<a href="https://colab.research.google.com/github/Kh-Harakeh/LSTM-based-Flood-Risk-Prediction-System/blob/main/LSTM_based_Flood_Risk_Prediction_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data Preparation**
We simulate synthetic daily data for several locations. Each location has features rainfall (mm), temperature (°C), humidity (%) over 100 days. We label a flood event (1) whenever rainfall exceeds a high threshold (else 0).

In [2]:
!pip install optuna
!pip install folium
!pip install torch torchvision scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import optuna
import shap
import folium

# Reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Define locations (latitude, longitude) for simulation
locations = [
    (18.86166345, 78.83558374),  # e.g., Hyderabad region
    (35.57071489, 77.65445079),  # e.g., Kashmir region
    (29.22782431, 73.10846346),  # e.g., Punjab region
    (25.36109604, 85.61073343),  # e.g., Bihar region
    (12.52454057, 81.82210065)   # e.g., Tamil Nadu region
]
n_loc = len(locations)
days = 100
dates = pd.date_range("2020-01-01", periods=days, freq='D')

# Simulate features for each location and day
rain = np.zeros((n_loc, days))
temp = np.zeros((n_loc, days))
hum  = np.zeros((n_loc, days))

for i in range(n_loc):
    # base seasonal rainfall (sinusoidal plus noise)
    base_rain = 50 * np.sin(np.linspace(0, 2*np.pi, days)) + 60
    rain[i] = base_rain + np.random.normal(scale=10, size=days)
    # temperature: warmer mid-period
    temp[i] = 30 + 5 * np.sin(np.linspace(0, 2*np.pi, days) - np.pi/4) + np.random.normal(scale=2, size=days)
    # humidity: correlated with rainfall
    hum[i] = 50 + 20 * np.sin(np.linspace(0, 2*np.pi, days) + np.pi/6) + np.random.normal(scale=5, size=days)

# Make sure no negative values
rain = np.clip(rain, 0, None)

# Determine flood label: 1 if rainfall exceeds 90th percentile threshold
rain_threshold = np.percentile(rain, 90)
flood_label = (rain >= rain_threshold).astype(int)

# Combine into a DataFrame for clarity (not used directly in model)
data_records = []
for i, (lat, lon) in enumerate(locations):
    for t in range(days):
        data_records.append({
            'date': dates[t],
            'latitude': lat,
            'longitude': lon,
            'rainfall': rain[i,t],
            'temperature': temp[i,t],
            'humidity': hum[i,t],
            'flood': flood_label[i,t]
        })
df = pd.DataFrame(data_records)
print(df.head())


        date   latitude  longitude   rainfall  temperature   humidity  flood
0 2020-01-01  18.861663  78.835584  64.967142    23.633725  61.788937      0
1 2020-01-02  18.861663  78.835584  61.788553    25.854531  63.882324      0
2 2020-01-03  18.861663  78.835584  72.806508    26.255053  67.527450      0
3 2020-01-04  18.861663  78.835584  84.692861    25.592907  68.366225      0
4 2020-01-05  18.861663  78.835584  70.215866    27.143155  57.141151      0


#**Creating Time-Series Sequences**
We train the LSTM to predict the next-day flood risk from the previous 7 days of weather. We construct input sequences of length 7 (past 7 days of features) and the label is the flood/no-flood on day 8. Each location contributes a series of samples.

In [4]:
# Sequence length (number of past days)
seq_len = 7

X = []
y = []
for i in range(n_loc):
    for start in range(days - seq_len):
        seq_features = np.stack([rain[i, start:start+seq_len],
                                  temp[i, start:start+seq_len],
                                  hum[i, start:start+seq_len]], axis=1)  # shape (seq_len, 3)
        X.append(seq_features)
        y.append(flood_label[i, start+seq_len])  # next day label

X = np.stack(X)  # shape (num_samples, seq_len, 3)
y = np.array(y)  # shape (num_samples,)
print(f"Constructed dataset: X={X.shape}, y={y.shape}, positive labels={y.sum()}")

# Split into training and test sets
dataset = TensorDataset(torch.tensor(X, dtype=torch.float32),
                        torch.tensor(y, dtype=torch.float32))
n_train = int(0.8 * len(dataset))
n_test = len(dataset) - n_train
train_dataset, test_dataset = random_split(dataset, [n_train, n_test])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=16)


Constructed dataset: X=(465, 7, 3), y=(465,), positive labels=50


#**LSTM Model Training**
We define a simple LSTM-based classifier in PyTorch. The model takes input shape (seq_len, features) and outputs a probability of flood. We use binary cross-entropy loss.

In [5]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=3, hidden_size=32, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.out = nn.Linear(hidden_size, 1)  # binary output
    def forward(self, x):
        # x: (batch, seq_len, features)
        h, _ = self.lstm(x)              # h: (batch, seq_len, hidden)
        out = self.out(h[:, -1, :])      # use last time-step's output
        return out.squeeze(1)            # (batch,)

# Train function
def train_model(model, loader, optimizer, criterion):
    model.train()
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

# Evaluation on test set (compute accuracy)
def eval_model(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in loader:
            logits = model(X_batch)
            preds = (torch.sigmoid(logits) >= 0.5).float()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)
    return correct / total

# Instantiate and train the model with default hyperparameters
model = LSTMClassifier(hidden_size=32)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(5):
    train_model(model, train_loader, optimizer, criterion)
acc = eval_model(model, test_loader)
print(f"Test accuracy (baseline model): {acc:.3f}")


Test accuracy (baseline model): 0.903


#**Hyperparameter Tuning with Optuna**
We use Optuna to tune the hidden size and learning rate. For each trial, we train for a few epochs and report test loss.

In [6]:
def objective(trial):
    # Suggest hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 8, 64)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-1)

    # Create model and optimizer
    model_t = LSTMClassifier(hidden_size=hidden_size)
    optimizer_t = optim.Adam(model_t.parameters(), lr=lr)
    criterion_t = nn.BCEWithLogitsLoss()

    # Quick train (e.g., 3 epochs for tuning)
    for _ in range(3):
        train_model(model_t, train_loader, optimizer_t, criterion_t)
    # Evaluate on validation (we use test_loader for simplicity)
    model_t.eval()
    losses = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            logits = model_t(X_batch)
            loss = criterion_t(logits, y_batch).item()
            losses.append(loss)
    # Return average loss
    return float(np.mean(losses))

# Run Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)
print("Best hyperparameters:", study.best_params)

# Train final model with best hyperparameters
best_hidden = study.best_params["hidden_size"]
best_lr = study.best_params["lr"]
model = LSTMClassifier(hidden_size=best_hidden)
optimizer = optim.Adam(model.parameters(), lr=best_lr)
for epoch in range(5):
    train_model(model, train_loader, optimizer, criterion)
print(f"Test accuracy (tuned model): {eval_model(model, test_loader):.3f}")


[I 2025-05-27 20:00:19,222] A new study created in memory with name: no-name-924f1c64-d577-4e09-8575-d0aeb8dfbd7b
<ipython-input-6-360ef443803d>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-4, 1e-1)
[I 2025-05-27 20:00:19,776] Trial 0 finished with value: 0.22638293852408728 and parameters: {'hidden_size': 47, 'lr': 0.0013332657831184793}. Best is trial 0 with value: 0.22638293852408728.
[I 2025-05-27 20:00:20,231] Trial 1 finished with value: 0.31654901305834454 and parameters: {'hidden_size': 16, 'lr': 0.09085405970733562}. Best is trial 0 with value: 0.22638293852408728.
[I 2025-05-27 20:00:21,052] Trial 2 finished with value: 0.27705761045217514 and parameters: {'hidden_size': 62, 'lr': 0.031351582612208564}. Best is trial 0 with value: 0.22638293852408728.
[I 2025-05-27 20:00:21,3

Best hyperparameters: {'hidden_size': 46, 'lr': 0.0018886932123276274}
Test accuracy (tuned model): 0.903


#**Feature Importance with SHAP**
We interpret the trained model using SHAP values. We use a small subset of training data as background for a KernelExplainer. SHAP values indicate how much each feature contributed to the model’s flood prediction.

In [8]:
seq_len = X_train_sample.shape[1]
n_features = X_train_sample.shape[2]

# 1. Prepare background and test arrays, flattened
flat_background = X_train_sample.reshape(50, -1)
flat_test       = X_test_sample.reshape(5, -1)

# 2. Define a flat-input prediction function
def predict_proba_flat(x_flat: np.ndarray) -> np.ndarray:
    """
    x_flat: shape (batch_size, seq_len * n_features)
    Returns flood probability: shape (batch_size,)
    """
    # reshape back to (batch, seq_len, n_features)
    x_seq = x_flat.reshape(-1, seq_len, n_features)
    x_tensor = torch.tensor(x_seq, dtype=torch.float32)
    with torch.no_grad():
        logits = model(x_tensor)                # (batch,)
        probs  = torch.sigmoid(logits).numpy()  # (batch,)
    return probs

# 3. Build the SHAP explainer on the flat background
explainer = shap.KernelExplainer(predict_proba_flat, flat_background)

# 4. Compute SHAP values on the flat test set
shap_values_flat = explainer.shap_values(flat_test, nsamples=100)
shap_values_flat = np.array(shap_values_flat)  # shape (5, 21)

print("SHAP values shape (samples, seq_len * features):", shap_values_flat.shape)


  0%|          | 0/5 [00:00<?, ?it/s]

SHAP values shape (samples, seq_len * features): (5, 21)


#**Visualization on Map**
Finally, we visualize the predicted flood probabilities for each location on a map. We take the last 7 days of data for each location and predict the probability of flood. The markers are sized by flood risk.


In [10]:
final_seq = []
for i in range(n_loc):
    seq = np.stack([rain[i, -seq_len:],
                    temp[i, -seq_len:],
                    hum[i, -seq_len:]], axis=1)
    final_seq.append(seq)
final_seq = np.stack(final_seq)  # (n_loc, seq_len, 3)
final_logits = model(torch.tensor(final_seq, dtype=torch.float32))
final_probs = torch.sigmoid(final_logits).detach().numpy()

# Create Folium map and add markers (with float casts):
m = folium.Map(location=[float(np.mean([lat for lat, _ in locations])),
                         float(np.mean([lon for _, lon in locations]))],
               zoom_start=6)

for (lat, lon), prob in zip(locations, final_probs):
    folium.CircleMarker(
        location=[float(lat), float(lon)],        # cast to native float
        radius=5 + float(prob) * 10,              # cast prob to float
        color='red' if prob > 0.5 else 'blue',
        fill=True,
        fill_opacity=0.7,
        popup=f"Flood Probability: {float(prob):.2f}"
    ).add_to(m)

m.save("predicted_flood_risk_map.html")
print("Map saved to predicted_flood_risk_map.html")

Map saved to predicted_flood_risk_map.html
